# Part 1

In [1]:
import re

source = open('12.txt').readlines()
lines = [x.strip().split(' ') for x in source]

def score_line(line):
    matches = re.findall("#+", line)
    if matches != None:
        score = [str(len(match)) for match in matches]
        return ','.join(score)
    else:
        return ''

line_matches = []
tests = 0
modifications = 0
for line in lines:
    qs = [i for i, x in enumerate(line[0]) if x == '?']
    matches = 0
    base = list(line[0])
    for masks in range(0, 2**len(qs)+1):
        test = base.copy()
        for i, q in enumerate(qs):
            if((masks & 2**i) > 0):
                test[q] = '#'
                modifications += 1
        score = score_line(''.join(test))
        tests += 1
        if(score == line[1]):
            matches += 1
    line_matches.append(matches)
sum(line_matches)

8075

# Part 2

## Make the five-x lines

In [2]:
import re
fivelines = []
for line in lines:
    source, target = line
    newsource = []
    newtarget = []
    for i in range(5):
        newsource.append(source)
        newtarget.append(target)
    outsource = '?'.join(newsource)
    outtarget = ','.join(newtarget)
    fivelines.append((outsource, outtarget))

## Define functions for analyzing positions, using a cache to memoize old position fetches

In [3]:
def line_combos(mask, scores):
    masklen = len(mask)
    maskMinDistance = (sum(scores) + len(scores)-1 - scores[-1])
    maxStart = masklen - maskMinDistance
    # grab the last score
    lastScore = scores[-1]
    # figure out the maximum start point
    pos = masklen - lastScore+1
    remask = "^[\?\.][#\?]{"+str(lastScore)+"}[\?\.]*$"
    startPrefix = "^[\?\.]*"
    positions = []
    for  i in range(pos, maskMinDistance-2, -1):

        match =  mask[i:]
        if re.match(remask, match):
            if(len(scores) == 1):
                startRemask = "^" + "[\?\.]{"+str(i+1)+"}" +remask[7:]
                # print("Checking again because this is the beginning", startRemask, mask, re.match(startRemask, match))
                if re.match(startRemask, mask) == None:
                    # print("not a start", mask, i)
                    continue
            # print("match", match, i)
            positions.append(i)
        # else:
            # if i == 0:
                # print("no match", match, i)
    return positions, maxStart, maskMinDistance, masklen

line_cache = {}
cache_hits = 0
def score_big_line(mask, scores):
    if(line_cache.get(mask + str(scores)) != None):
        # print("Using Cache!!!")
        # cache_hits = cache_hits + 1
        return line_cache[mask + str(scores)]
    ret = line_combos(mask, scores)
    # if ret == None: 
    #     print("No combos")
    #     return 0
    output = 0
    for start in ret[0]:
        shorter_mask = mask[0: start] + "."
        shorter_score = scores[0:-1] 
        if len(shorter_score) == 0:
            output += 1
        else:
            output += score_big_line(shorter_mask, shorter_score)
    line_cache[mask + str(scores)] = output
    return output

## Re-implementing part 1 using the new technique. 

In [4]:
results = []
for testIndex, testLine in enumerate(lines):
    result = score_big_line("." + testLine[0],[int(x) for x in testLine[1].split(',')]
    )
    # print(testIndex, "combos",result, line_matches[testIndex], result == line_matches[testIndex] )
    results.append(result)
sum(results)

8075

## Test to see if the two techniques generate different results

In [5]:
for(i, line) in enumerate(lines):
    if results[i] != line_matches[i]:
        print("No match", i, line, results[i], line_matches[i])

## Output using 5x lines

In [6]:
cache_hits = 0
results = []
fivetargets = [
    1,16384,1,16,2500,506250
]
for testIndex, testLine in enumerate(fivelines):
    # print("line " + str(testIndex), testLine)
    result = score_big_line("." + testLine[0],[int(x) for x in testLine[1].split(',')]
    )
    # print(testIndex, "combos",result )
    results.append(result)
sum(results)

4232520187524